<a href="https://colab.research.google.com/github/gemstuition/END_Program/blob/main/Assignment_session4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import torch
import torch.nn as nn
from torchtext import data


SEED = 1234

torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

TEXT = data.Field(tokenize = 'spacy', include_lengths = True)
LABEL = data.LabelField(dtype = torch.float)

In [5]:
from torchtext import datasets

train_data, test_data = datasets.IMDB.splits(TEXT, LABEL)


In [6]:
import random

train_data, valid_data = train_data.split(random_state = random.seed(SEED))

In [8]:
for i in range(len(train_data)):
  vars(train_data.examples[i]).get('text').reverse()

In [9]:
MAX_VOCAB_SIZE = 25000

TEXT.build_vocab(train_data, 
                 max_size = MAX_VOCAB_SIZE, 
                 vectors = "glove.6B.100d", 
                 unk_init = torch.Tensor.normal_)

LABEL.build_vocab(train_data)

.vector_cache/glove.6B.zip: 862MB [06:32, 2.20MB/s]                          
 99%|█████████▉| 397767/400000 [00:16<00:00, 25326.87it/s]

In [10]:
BATCH_SIZE = 64

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE,
    sort_within_batch = True,
    device = device)

In [11]:
class RNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, 
                 bidirectional, dropout, pad_idx):
        
        super().__init__()
        self.factor = 2 if bidirectional else 1
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx = pad_idx)
        self.rnn = nn.ModuleList([
                                  nn.LSTM(embedding_dim, hidden_dim * 2, bidirectional = bidirectional),
                                  nn.Dropout(dropout),
                                  nn.LSTM(hidden_dim * 2 * self.factor, hidden_dim, bidirectional = bidirectional),
                                  nn.LSTM(hidden_dim * self.factor, hidden_dim, bidirectional = bidirectional),
        ])
        self.fc = nn.Linear(hidden_dim * self.factor, output_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, text, text_lengths):

        embedded = self.dropout(self.embedding(text))
                
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, text_lengths)
        for layer in self.rnn:
            if not isinstance(layer, torch.nn.modules.dropout.Dropout):
                packed_embedded, (hidden, cell) = layer(packed_embedded)
            else:
                packed_embedded, packed_lengths = nn.utils.rnn.pad_packed_sequence(packed_embedded)
                packed_embedded = nn.utils.rnn.pack_padded_sequence(layer(packed_embedded), packed_lengths)
 
        hidden = self.dropout(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1)) if self.factor == 2 else self.dropout(hidden[-1,:,:])
        return self.fc(hidden)

In [12]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 100
HIDDEN_DIM = 64
OUTPUT_DIM = 1
BIDIRECTIONAL = True
DROPOUT = 0.2
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

model = RNN(INPUT_DIM, 
            EMBEDDING_DIM, 
            HIDDEN_DIM, 
            OUTPUT_DIM, 
            BIDIRECTIONAL, 
            DROPOUT, 
            PAD_IDX).to(device)

In [13]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 3,000,041 trainable parameters


In [14]:
pretrained_embeddings = TEXT.vocab.vectors

print(pretrained_embeddings.shape)

torch.Size([25002, 100])


In [15]:
model.embedding.weight.data.copy_(pretrained_embeddings)

tensor([[-0.1117, -0.4966,  0.1631,  ...,  1.2647, -0.2753, -0.1325],
        [-0.8555, -0.7208,  1.3755,  ...,  0.0825, -1.1314,  0.3997],
        [-0.0382, -0.2449,  0.7281,  ..., -0.1459,  0.8278,  0.2706],
        ...,
        [-0.1830, -1.1160,  2.3601,  ..., -1.6465,  0.2431,  0.5666],
        [-0.2326, -1.2659, -1.1949,  ...,  0.9952, -0.0324, -0.1769],
        [-0.6184,  0.0806,  1.7381,  ...,  0.0445, -0.5604, -0.3227]],
       device='cuda:0')

In [16]:

UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]

model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)

print(model.embedding.weight.data)

tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [-0.0382, -0.2449,  0.7281,  ..., -0.1459,  0.8278,  0.2706],
        ...,
        [-0.1830, -1.1160,  2.3601,  ..., -1.6465,  0.2431,  0.5666],
        [-0.2326, -1.2659, -1.1949,  ...,  0.9952, -0.0324, -0.1769],
        [-0.6184,  0.0806,  1.7381,  ...,  0.0445, -0.5604, -0.3227]],
       device='cuda:0')


In [17]:
import torch.optim as optim

optimizer = optim.Adam(model.parameters())

In [18]:

criterion = nn.BCEWithLogitsLoss()

model = model.to(device)
criterion = criterion.to(device)

In [19]:
def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """
    #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum() / len(correct)
    return acc

In [20]:
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0

    model.train()
    
    for batch in iterator:
        optimizer.zero_grad()
        text, text_lengths = batch.text
        text_lengths = text_lengths.cpu()
        predictions = model(text, text_lengths).squeeze(1)
        loss = criterion(predictions, batch.label)
        acc = binary_accuracy(predictions, batch.label)
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [21]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
        for batch in iterator:
            text, text_lengths = batch.text
            text_lengths = text_lengths.cpu()
            predictions = model(text, text_lengths).squeeze(1)
            loss = criterion(predictions, batch.label)
            acc = binary_accuracy(predictions, batch.label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [22]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs


In [28]:
N_EPOCHS = 12
best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut2-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 0m 34s
	Train Loss: 0.216 | Train Acc: 91.80%
	 Val. Loss: 0.308 |  Val. Acc: 87.56%
Epoch: 02 | Epoch Time: 0m 34s
	Train Loss: 0.177 | Train Acc: 93.54%
	 Val. Loss: 0.295 |  Val. Acc: 88.00%
Epoch: 03 | Epoch Time: 0m 34s
	Train Loss: 0.149 | Train Acc: 94.88%
	 Val. Loss: 0.315 |  Val. Acc: 88.53%
Epoch: 04 | Epoch Time: 0m 34s
	Train Loss: 0.115 | Train Acc: 96.11%
	 Val. Loss: 0.331 |  Val. Acc: 88.04%
Epoch: 05 | Epoch Time: 0m 33s
	Train Loss: 0.095 | Train Acc: 96.92%
	 Val. Loss: 0.358 |  Val. Acc: 87.50%
Epoch: 06 | Epoch Time: 0m 33s
	Train Loss: 0.078 | Train Acc: 97.57%
	 Val. Loss: 0.396 |  Val. Acc: 88.17%
Epoch: 07 | Epoch Time: 0m 34s
	Train Loss: 0.064 | Train Acc: 98.00%
	 Val. Loss: 0.399 |  Val. Acc: 87.96%
Epoch: 08 | Epoch Time: 0m 33s
	Train Loss: 0.054 | Train Acc: 98.44%
	 Val. Loss: 0.402 |  Val. Acc: 87.69%
Epoch: 09 | Epoch Time: 0m 32s
	Train Loss: 0.045 | Train Acc: 98.71%
	 Val. Loss: 0.436 |  Val. Acc: 87.78%
Epoch: 10 | Epoch T

In [33]:
model.load_state_dict(torch.load('tut2-model.pt'))

test_loss, test_acc = evaluate(model, test_iterator, criterion)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

Test Loss: 0.329 | Test Acc: 86.67%


In [34]:
import spacy
nlp = spacy.load('en')

def predict_sentiment(model, sentence):
    model.eval()
    tokenized = [tok.text for tok in nlp.tokenizer(sentence)]
    indexed = [TEXT.vocab.stoi[t] for t in tokenized]
    length = [len(indexed)]
    tensor = torch.LongTensor(indexed).to(device)
    tensor = tensor.unsqueeze(1)
    length_tensor = torch.LongTensor(length)
    prediction = torch.sigmoid(model(tensor, length_tensor))
    return prediction.item()

In [35]:
predict_sentiment(model, "This film is terrible")

0.02736903727054596

In [36]:
predict_sentiment(model, "This film is great")

0.8829988241195679